# TradPal - BTC/USDT Backtest Example

This notebook demonstrates how to use TradPal for backtesting trading strategies on BTC/USDT.

## Overview
- Load historical BTC/USDT data
- Calculate technical indicators
- Generate trading signals
- Run backtest simulation
- Analyze performance metrics

## Requirements
- TradPal installed
- Jupyter notebook environment
- Internet connection for data fetching

In [ ]:
# Import required libraries
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Add TradPal to path
project_root = os.path.abspath('..')
sys.path.append(project_root)

# Import TradPal modules
from src.data_fetcher import fetch_historical_data
from src.indicators import calculate_indicators
from src.signal_generator import generate_signals
from src.risk_management import calculate_risk_management
from src.backtester import run_backtest

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully")
print(f"📊 Working directory: {os.getcwd()}")

## Step 1: Configure Trading Parameters

Set up the trading parameters for our BTC/USDT backtest.

In [ ]:
# Trading configuration
CONFIG = {
    'symbol': 'BTC/USDT',
    'exchange': 'kraken',
    'timeframe': '1h',  # 1-hour candles for faster testing
    'start_date': '2024-01-01',
    'end_date': '2024-06-01',  # 6 months of data
    
    # Indicator parameters
    'ema_short': 9,
    'ema_long': 21,
    'rsi_period': 14,
    'rsi_oversold': 30,
    'rsi_overbought': 70,
    'bb_period': 20,
    'bb_std_dev': 2.0,
    
    # Risk management
    'initial_capital': 10000.0,  # $10,000 starting capital
    'risk_per_trade': 0.01,  # 1% risk per trade
    'max_position_size': 0.1,  # Max 10% of capital per position
    
    # Trading fees (Kraken spot)
    'maker_fee': 0.0016,  # 0.16%
    'taker_fee': 0.0026,  # 0.26%
}

print("⚙️ Trading configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## Step 2: Fetch Historical Data

Download historical BTC/USDT data from Kraken exchange.

In [ ]:
print(f"📥 Fetching {CONFIG['symbol']} data from {CONFIG['exchange']}...")
print(f"   Period: {CONFIG['start_date']} to {CONFIG['end_date']}")
print(f"   Timeframe: {CONFIG['timeframe']}")

# Fetch historical data
try:
    data = fetch_historical_data(
        symbol=CONFIG['symbol'],
        exchange=CONFIG['exchange'],
        timeframe=CONFIG['timeframe'],
        start_date=CONFIG['start_date'],
        end_date=CONFIG['end_date']
    )
    
    print(f"✅ Data fetched successfully: {len(data)} candles")
    print(f"   Date range: {data.index[0]} to {data.index[-1]}")
    print(f"   Price range: ${data['close'].min():.2f} - ${data['close'].max():.2f}")
    
except Exception as e:
    print(f"❌ Error fetching data: {e}")
    print("💡 Make sure you have internet connection and valid API credentials")
    raise

## Step 3: Calculate Technical Indicators

Apply technical indicators to the price data.

In [ ]:
print("📊 Calculating technical indicators...")

# Create indicator configuration
indicator_config = {
    'ema': {
        'enabled': True,
        'periods': [CONFIG['ema_short'], CONFIG['ema_long']]
    },
    'rsi': {
        'enabled': True,
        'period': CONFIG['rsi_period'],
        'oversold': CONFIG['rsi_oversold'],
        'overbought': CONFIG['rsi_overbought']
    },
    'bb': {
        'enabled': True,
        'period': CONFIG['bb_period'],
        'std_dev': CONFIG['bb_std_dev']
    },
    'atr': {
        'enabled': True,
        'period': 14
    }
}

# Calculate indicators
try:
    data_with_indicators = calculate_indicators(data, config=indicator_config)
    print("✅ Indicators calculated successfully")
    print(f"   Available columns: {len(data_with_indicators.columns)}")
    
    # Show sample of calculated indicators
    indicator_columns = [col for col in data_with_indicators.columns if col not in ['open', 'high', 'low', 'close', 'volume']]
    print(f"   Indicator columns: {indicator_columns[:5]}...")  # Show first 5
    
except Exception as e:
    print(f"❌ Error calculating indicators: {e}")
    raise

## Step 4: Generate Trading Signals

Apply the trading strategy to generate buy/sell signals.

In [ ]:
print("🎯 Generating trading signals...")

# Generate signals
try:
    data_with_signals = generate_signals(data_with_indicators)
    print("✅ Signals generated successfully")
    
    # Count signals
    buy_signals = data_with_signals['Buy_Signal'].sum()
    sell_signals = data_with_signals['Sell_Signal'].sum()
    
    print(f"🟢 Buy signals: {buy_signals}")
    print(f"🔴 Sell signals: {sell_signals}")
    print(f"📊 Signal density: {(buy_signals + sell_signals) / len(data_with_signals) * 100:.2f}%")
    
except Exception as e:
    print(f"❌ Error generating signals: {e}")
    raise

## Step 5: Apply Risk Management

Calculate position sizes, stop losses, and take profits.

In [ ]:
print("⚠️ Applying risk management...")

# Apply risk management
try:
    data_complete = calculate_risk_management(data_with_signals)
    print("✅ Risk management applied successfully")
    
    # Show risk management sample
    risk_sample = data_complete[['Position_Size_Percent', 'Stop_Loss_Buy', 'Take_Profit_Buy', 'Leverage']].tail(5)
    print("\n📋 Recent risk management parameters:")
    print(risk_sample)
    
except Exception as e:
    print(f"❌ Error applying risk management: {e}")
    raise

## Step 6: Visualize the Data

Create charts to visualize the price action, indicators, and signals.

In [ ]:
print("📈 Creating visualizations...")

# Create a comprehensive chart
fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)
fig.suptitle(f'{CONFIG["symbol"]} - Technical Analysis & Signals', fontsize=16)

# Plot 1: Price and EMAs
axes[0].plot(data_complete.index, data_complete['close'], label='Close Price', color='black', alpha=0.7)
axes[0].plot(data_complete.index, data_complete['EMA9'], label=f'EMA{CONFIG["ema_short"]}', color='blue')
axes[0].plot(data_complete.index, data_complete['EMA21'], label=f'EMA{CONFIG["ema_long"]}', color='red')
axes[0].set_title('Price & Moving Averages')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot buy/sell signals on price chart
buy_signals_data = data_complete[data_complete['Buy_Signal'] == 1]
sell_signals_data = data_complete[data_complete['Sell_Signal'] == 1]

if not buy_signals_data.empty:
    axes[0].scatter(buy_signals_data.index, buy_signals_data['close'], 
                    marker='^', color='green', s=100, label='Buy Signal', zorder=5)
if not sell_signals_data.empty:
    axes[0].scatter(sell_signals_data.index, sell_signals_data['close'], 
                    marker='v', color='red', s=100, label='Sell Signal', zorder=5)

# Plot 2: RSI
axes[1].plot(data_complete.index, data_complete['RSI'], color='purple')
axes[1].axhline(y=CONFIG['rsi_overbought'], color='red', linestyle='--', alpha=0.7, label='Overbought')
axes[1].axhline(y=CONFIG['rsi_oversold'], color='green', linestyle='--', alpha=0.7, label='Oversold')
axes[1].axhline(y=50, color='gray', linestyle='-', alpha=0.5)
axes[1].set_title('RSI Indicator')
axes[1].set_ylim(0, 100)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Bollinger Bands
axes[2].plot(data_complete.index, data_complete['close'], label='Close', color='black', alpha=0.7)
axes[2].plot(data_complete.index, data_complete['BB_middle'], label='BB Middle', color='orange')
axes[2].fill_between(data_complete.index, data_complete['BB_lower'], data_complete['BB_upper'], 
                    alpha=0.2, color='orange', label='BB Range')
axes[2].set_title('Bollinger Bands')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualization created successfully")

## Step 7: Run Backtest Simulation

Execute the backtest to evaluate the trading strategy performance.

In [ ]:
print("🏃 Running backtest simulation...")

# Run backtest
try:
    backtest_results = run_backtest(
        symbol=CONFIG['symbol'],
        timeframe=CONFIG['timeframe'],
        start_date=CONFIG['start_date'],
        end_date=CONFIG['end_date'],
        initial_capital=CONFIG['initial_capital'],
        risk_per_trade=CONFIG['risk_per_trade'],
        maker_fee=CONFIG['maker_fee'],
        taker_fee=CONFIG['taker_fee']
    )
    
    print("✅ Backtest completed successfully")
    
    # Extract key metrics
    if 'backtest_results' in backtest_results:
        metrics = backtest_results['backtest_results']
        
        print("\n📊 Backtest Results:")
        print("=" * 50)
        print(f"Total Trades: {metrics.get('total_trades', 0)}")
        print(f"Win Rate: {metrics.get('win_rate', 0):.1f}%")
        print(f"Total P&L: ${metrics.get('total_pnl', 0):.2f}")
        print(f"Max Drawdown: {metrics.get('max_drawdown', 0):.2f}%")
        print(f"Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.2f}")
        print(f"Final Capital: ${metrics.get('final_capital', 0):.2f}")
        
        # Calculate additional metrics
        total_return = (metrics.get('final_capital', CONFIG['initial_capital']) - CONFIG['initial_capital']) / CONFIG['initial_capital'] * 100
        print(f"Total Return: {total_return:.2f}%")
        
        if metrics.get('total_trades', 0) > 0:
            avg_trade = metrics.get('total_pnl', 0) / metrics.get('total_trades', 0)
            print(f"Average P&L per Trade: ${avg_trade:.2f}")
    else:
        print("⚠️ No backtest results found")
        
except Exception as e:
    print(f"❌ Error running backtest: {e}")
    print("💡 Check that all required modules are available and data is loaded")

## Step 8: Performance Analysis

Analyze the backtest results and create performance visualizations.

In [ ]:
print("📈 Analyzing performance...")

# Create performance visualization if backtest was successful
if 'backtest_results' in backtest_results and backtest_results['backtest_results']:
    metrics = backtest_results['backtest_results']
    
    # Create performance dashboard
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Backtest Performance Analysis', fontsize=16)
    
    # Performance metrics
    metrics_data = {
        'Win Rate': f"{metrics.get('win_rate', 0):.1f}%",
        'Total P&L': f"${metrics.get('total_pnl', 0):.2f}",
        'Max Drawdown': f"{metrics.get('max_drawdown', 0):.2f}%",
        'Sharpe Ratio': f"{metrics.get('sharpe_ratio', 0):.2f}",
        'Total Trades': metrics.get('total_trades', 0),
        'Final Capital': f"${metrics.get('final_capital', 0):.2f}"
    }
    
    # Plot metrics as text
    axes[0, 0].text(0.1, 0.5, '\n'.join([f'{k}: {v}' for k, v in metrics_data.items()]), 
                   fontsize=12, verticalalignment='center')
    axes[0, 0].set_title('Performance Metrics')
    axes[0, 0].set_xlim(0, 1)
    axes[0, 0].set_ylim(0, 1)
    axes[0, 0].axis('off')
    
    # Placeholder for equity curve (would need actual trade data)
    axes[0, 1].plot([CONFIG['initial_capital'], metrics.get('final_capital', CONFIG['initial_capital'])], 
                   marker='o', linewidth=2, markersize=8)
    axes[0, 1].set_title('Equity Curve (Simplified)')
    axes[0, 1].set_ylabel('Capital ($)')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Win/Loss distribution placeholder
    if metrics.get('total_trades', 0) > 0:
        win_rate = metrics.get('win_rate', 0)
        loss_rate = 100 - win_rate
        axes[1, 0].bar(['Wins', 'Losses'], [win_rate, loss_rate], color=['green', 'red'])
        axes[1, 0].set_title('Win/Loss Distribution')
        axes[1, 0].set_ylabel('Percentage (%)')
        
        for i, v in enumerate([win_rate, loss_rate]):
            axes[1, 0].text(i, v + 1, f'{v:.1f}%', ha='center')
    
    # Risk metrics
    risk_data = {
        'Max Drawdown': metrics.get('max_drawdown', 0),
        'Sharpe Ratio': metrics.get('sharpe_ratio', 0),
        'Risk per Trade': CONFIG['risk_per_trade'] * 100
    }
    
    axes[1, 1].bar(risk_data.keys(), risk_data.values(), color='orange')
    axes[1, 1].set_title('Risk Metrics')
    axes[1, 1].set_ylabel('Value')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Performance analysis completed")
    
else:
    print("⚠️ No backtest results available for analysis")

This notebook demonstrated a complete BTC/USDT backtest workflow using TradPal: